# Download Data Sweden

In [1]:
%matplotlib widget
import matplotlib.pylab as plt
import seaborn as sns

import numpy as np

import pandas as pd

import re

from tqdm import tqdm
import glob
import os

from owslib.wms import WebMapService
import rasterio

## Floading maps

All maps for Sweden: https://gisapp.msb.se/Apps/oversvamningsportal/hemta-data.html

Right now I'll be using: https://gisapp.msb.se/arcgis/services/Oversvamningskarteringar/karteringar/MapServer/WmsServer

## Check what layers are available

In [2]:
wms = WebMapService('https://gisapp.msb.se/arcgis/services/Oversvamningskarteringar/karteringar/MapServer/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

0      Översiktliga, beräknat högsta flöde ['EPSG:4326', 'EPSG:3006']
1               Översiktliga, 100-årsflöde ['EPSG:4326', 'EPSG:3006']
2  Tvärsektioner, översiktliga karteringar ['EPSG:4326', 'EPSG:3006']
3         Torne älv, beräknat högsta flöde ['EPSG:4326', 'EPSG:3006']
4                  Torne älv, 250-årsflöde ['EPSG:4326', 'EPSG:3006']
5                  Torne älv, 100-årsflöde ['EPSG:4326', 'EPSG:3006']
6                 Torne älv, tvärsektioner ['EPSG:4326', 'EPSG:3006']
7 Göta älv, 1400 m³/s (Motsvarande beräknat högsta flöde) ['EPSG:4326', 'EPSG:3006']
8 Göta älv, 1200 m³/s (Motsvarande 200-årsflöde) ['EPSG:4326', 'EPSG:3006']
9 Göta älv, 1030 m³/s (Motsvarande 100-årsflöde) ['EPSG:4326', 'EPSG:3006']
10                  Göta älv, tvärsektioner ['EPSG:4326', 'EPSG:3006']
11                    Beräknat högsta flöde ['EPSG:4326', 'EPSG:3006']
12 200-årsflöde (Klimatanpassat flöde för slutet av seklet) ['EPSG:4326', 'EPSG:3006']
13 100-årsflöde (Klimatanpassat flöde för sl

## Download data and save it with georeferences

Not sure what all of these are, so I'm downloading all.

In [24]:
def save_file(wms,layer,size,crs,dir_path):
    
    srs = wms[layer].crsOptions
    bbox = wms[layer].boundingBoxWGS84 
    output_name = wms[layer].title
    output_name = output_name.replace(',','').replace('(','').replace(')','').replace(' ','_').replace('m³/s','m3s')
    
    if crs not in srs:
        print('CRS not available (available options:)',srs)
        pass
        
    # Get the image from the server
    img = wms.getmap(layers=[layer],
                     styles=['default'],
                     srs=crs,
                     bbox=bbox,
                     size=size,
                     transparent=True,
                     format='image/png8'
                    )
    
    # Save the array as an image
    print('Saved', dir_path+output_name+'.png')
    with open(dir_path+output_name+'.png', 'wb') as out:
        out.write(img.read())
       
    # Re-open the same image in rasterio to georeference it
    img_array = rasterio.open(dir_path+output_name+'.png').read().squeeze() 
    img_shape = img_array.shape
    dst_transform = rasterio.transform.from_bounds(west = bbox[0],
                                                   south = bbox[1],
                                                   east = bbox[2],
                                                   north = bbox[3],
                                                   width=img_shape[1], height=img_shape[0])
    # Write it out to a file.
    with rasterio.open(dir_path+output_name+'.png', 'w', driver='GTiff',
                       width=img_shape[1], height=img_shape[0],
                       count=1, dtype=np.uint8, nodata=0,
                       transform=dst_transform, crs=crs,
                        compress='lzw',) as dst:
        dst.write(img_array.astype(np.uint8), indexes=1)

In [25]:
dir_path = 'SWE/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
        
for layer_nb in wms.contents:
    save_file(wms,
              layer=layer_nb,
              size=(4000,4000),
              crs='EPSG:4326',
              dir_path=dir_path)

Saved SWE/Översiktliga_beräknat_högsta_flöde.png
Saved SWE/Översiktliga_100-årsflöde.png
Saved SWE/Tvärsektioner_översiktliga_karteringar.png
Saved SWE/Torne_älv_beräknat_högsta_flöde.png
Saved SWE/Torne_älv_250-årsflöde.png
Saved SWE/Torne_älv_100-årsflöde.png
Saved SWE/Torne_älv_tvärsektioner.png
Saved SWE/Göta_älv_1400_m3s_Motsvarande_beräknat_högsta_flöde.png
Saved SWE/Göta_älv_1200_m3s_Motsvarande_200-årsflöde.png
Saved SWE/Göta_älv_1030_m3s_Motsvarande_100-årsflöde.png
Saved SWE/Göta_älv_tvärsektioner.png
Saved SWE/Beräknat_högsta_flöde.png
Saved SWE/200-årsflöde_Klimatanpassat_flöde_för_slutet_av_seklet.png
Saved SWE/100-årsflöde_Klimatanpassat_flöde_för_slutet_av_seklet.png
Saved SWE/Tvärsektioner.png
Saved SWE/Översvämningskarterade_vattendrag_översikt.png
